<a href="https://colab.research.google.com/github/pvbernhard/recuperacao_informacao/blob/master/ativ_eng_info_lol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geral

## Bibliotecas

In [ ]:
from typing import List, Dict, Any, Optional, Tuple, Union

## Funções

In [ ]:
def cria_diretorios(dirs: List[str]) -> None:
  from os import makedirs

  for d in dirs:
    makedirs(d, exist_ok=True)

## Variáveis

In [ ]:
BASE_FOLDER = 'league_of_legends'
REPRESENTACAO_FOLDER = 'representacao'

cria_diretorios([BASE_FOLDER, REPRESENTACAO_FOLDER])

# Etapa 1: Aquisição e preparação dos documentos

## Bibliotecas

## Funções

In [ ]:
!pip -q install kora

     |████████████████████████████████| 57 kB 3.8 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 


In [ ]:
VERSAO = '11.16.1'


def get_historia_universo(id: str) -> str:
  from kora.selenium import wd as webdriver
  import time, re

  url = f'https://universe.leagueoflegends.com/pt_BR/story/champion/{id}/'
  webdriver.get(url)

  paragrafos = None
  contador = 0
  while not bool(paragrafos):
    contador += 1
    if contador > 10:
      raise Exception(f'Erro: página {id} não está carregando.')
    time.sleep(1)
    paragrafos = webdriver.find_element_by_id('CatchElement')
  
  # substitui <br>
  texto = re.sub(r'(<br>)+', '\n', str(paragrafos))
  # substitui paragrafos por \n
  texto = re.sub(r'<p[ \w=\"]*>([^<]+)<\/\w+>', '\g<1>\n', texto)
  # remove tags
  texto = re.sub(r'<\/*[ \w=\"]+>', '', texto)
  # fix ponto continuando
  texto = re.sub(r'\.(\w)', '. \g<1>', texto)

  return texto


def get_campeoes_ids(versao:str = VERSAO) -> List[str]:
  import requests

  resposta = requests.get(f"http://ddragon.leagueoflegends.com/cdn/{versao}/"
                          f"data/pt_BR/champion.json").json()
  campeoes = resposta.get('data')
  campeoes_ids = []
  for nome in campeoes:
    campeoes_ids.append(nome)
  return campeoes_ids


def get_campeao_dados(id: str,
                      versao:str = VERSAO) -> Dict[str, Any]:
  import requests

  resposta = requests.get(f"http://ddragon.leagueoflegends.com/cdn/{versao}/"
                          f"data/pt_BR/champion/{id}.json").json()
  return resposta.get('data').get(id)


def gerar_arquivo(campeao: Dict[str, Any],
                  diretorio: str,
                  versao:str = VERSAO) -> None:
  import os, re

  nome = campeao.get('name')
  titulo = campeao.get('title')
  imagem = f"http://ddragon.leagueoflegends.com/cdn/{versao}/img/"\
           f"champion/{campeao.get('id')}.png"
  lore = campeao.get('lore')
  lista_tags = campeao.get('tags')

  traducao_tags = {
      'Tank': 'Tanque',
      'Fighter': 'Lutador',
      'Assassin': 'Assassino',
      'Mage': 'Mago',
      'Support': 'Suporte',
      'Marksman': 'Atirador'
  }
  lista_tags_pt = []
  for tag in lista_tags:
    lista_tags_pt.append(traducao_tags.get(tag))

  tags = ''
  for tag in lista_tags_pt:
    tags += tag + ', '
  tags = tags[:-2]

  passiva_dados = campeao.get('passive')
  passiva = {
      'nome': passiva_dados.get('name'),
      'descricao': passiva_dados.get('description'),
      'imagem': f"http://ddragon.leagueoflegends.com/cdn/{versao}/img/"\
                f"passive/{passiva_dados.get('image').get('full')}"
  }
  # remove tags
  passiva['descricao'] = re.sub(r'<\/*[ \w=\"]+>', '', passiva.get('descricao'))
  # fix ponto continuando
  passiva['descricao'] = re.sub(r'\.(\w)', '. \g<1>', passiva.get('descricao'))

  habilidades = []
  for habilidade in campeao.get('spells'):
    _descricao = habilidade.get('description')

    # remove tags
    _descricao = re.sub(r'<\/*[ \w=\"]+>', '', _descricao)
    # fix ponto continuando
    _descricao = re.sub(r'\.(\w)', '. \g<1>', _descricao)

    habilidades.append({
        'nome': habilidade.get('name'),
        'descricao': _descricao,
        'imagem': f"http://ddragon.leagueoflegends.com/cdn/{versao}/img/"\
                  f"spell/{habilidade.get('image').get('full')}"
    })

  historia_completa = get_historia_universo(campeao.get('id'))

  print(f"História de {campeao.get('id')}:", str(len(historia_completa)))

  with open(os.path.join(diretorio, f"{campeao.get('id')}.txt"), mode='w+') as arquivo:
    print(nome + ', ' + titulo, file=arquivo)
    print(tags, file=arquivo)
    print(imagem, file=arquivo)

    print('', file=arquivo)
    print(passiva.get('nome'), file=arquivo)
    print(passiva.get('descricao'), file=arquivo)
    print(passiva.get('imagem'), file=arquivo)
    for habilidade in habilidades:
      print('', file=arquivo)
      print(habilidade.get('nome'), file=arquivo)
      print(habilidade.get('descricao'), file=arquivo)
      print(habilidade.get('imagem'), file=arquivo)
    
    print('', file=arquivo)
    print(lore, file=arquivo)
    print('', file=arquivo)
    print(historia_completa, file=arquivo)

    print(f"Arquivo {campeao.get('id')} gerado.")


## Gera arquivos

In [ ]:
campeoes_ids = get_campeoes_ids()
contador = 0
for id in campeoes_ids:
  campeao_dados = get_campeao_dados(id)
  gerar_arquivo(campeao_dados, BASE_FOLDER)
  contador += 1
print(f'Finalizado. ({contador})')

História de Aatrox: 4370
Arquivo Aatrox gerado.
História de Ahri: 3602
Arquivo Ahri gerado.
História de Akali: 3819
Arquivo Akali gerado.
História de Akshan: 4814
Arquivo Akshan gerado.
História de Alistar: 4727
Arquivo Alistar gerado.
História de Amumu: 4643
Arquivo Amumu gerado.
História de Anivia: 4429
Arquivo Anivia gerado.
História de Annie: 3964
Arquivo Annie gerado.
História de Aphelios: 4925
Arquivo Aphelios gerado.
História de Ashe: 4293
Arquivo Ashe gerado.
História de AurelionSol: 4240
Arquivo AurelionSol gerado.
História de Azir: 12034
Arquivo Azir gerado.
História de Bard: 3384
Arquivo Bard gerado.
História de Blitzcrank: 4358
Arquivo Blitzcrank gerado.
História de Brand: 3201
Arquivo Brand gerado.
História de Braum: 4825
Arquivo Braum gerado.
História de Caitlyn: 4092
Arquivo Caitlyn gerado.
História de Camille: 5520
Arquivo Camille gerado.
História de Cassiopeia: 4738
Arquivo Cassiopeia gerado.
História de Chogath: 1357
Arquivo Chogath gerado.
História de Corki: 1687
Arq

### Gera zip

In [ ]:
!zip -r {BASE_FOLDER}.zip {BASE_FOLDER}

  adding: league_of_legends/ (stored 0%)
  adding: league_of_legends/Ekko.txt (deflated 55%)
  adding: league_of_legends/TahmKench.txt (deflated 56%)
  adding: league_of_legends/Lucian.txt (deflated 56%)
  adding: league_of_legends/Fiora.txt (deflated 55%)
  adding: league_of_legends/Kindred.txt (deflated 57%)
  adding: league_of_legends/Pantheon.txt (deflated 57%)
  adding: league_of_legends/Samira.txt (deflated 54%)
  adding: league_of_legends/Jinx.txt (deflated 53%)
  adding: league_of_legends/Jax.txt (deflated 55%)
  adding: league_of_legends/Riven.txt (deflated 54%)
  adding: league_of_legends/Gwen.txt (deflated 55%)
  adding: league_of_legends/MasterYi.txt (deflated 54%)
  adding: league_of_legends/Yorick.txt (deflated 58%)
  adding: league_of_legends/Diana.txt (deflated 55%)
  adding: league_of_legends/Khazix.txt (deflated 57%)
  adding: league_of_legends/Alistar.txt (deflated 54%)
  adding: league_of_legends/Corki.txt (deflated 53%)
  adding: league_of_legends/Aphelios.txt (def

# Etapa 2: Indexação e armazenamento dos documentos

## Bibliotecas

In [ ]:
import os
import pandas as pd

import nltk


nltk.download('stopwords')
nltk.download('punkt')

!pip install -q unidecode

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 241 kB 7.2 MB/s 


## Funções

In [ ]:
def gera_tokens(texto: str = '',
                palavras: List[str] = None, 
                linguagem: str = 'portuguese',
                no_stemming: bool = False) -> Optional[List[str]]:
  """
  Gera tokens baseado em um texto.
  """
  from nltk.corpus import stopwords
  from nltk.tokenize import word_tokenize
  from nltk.stem.snowball import SnowballStemmer
  from unidecode import unidecode

  tokens = None

  if len(texto) > 0:
    # gera stop words
    stop_words = set(stopwords.words(linguagem))

    # tokeniza o texto
    tokens = word_tokenize(texto)

    # remove palavras com tamanho < 3
    # remove pontuações (strip)
    # transforma em caixa baixa (lower)
    tokens = [t.lower() for t in tokens if len(t) >= 3 and t.strip()]

    # remove stop words
    tokens = [t for t in tokens if not t.lower() in stop_words]

    # stemming
    if not no_stemming:
      stemmer = SnowballStemmer(language=linguagem)
      tokens = [stemmer.stem(t) for t in tokens]

    # remove acentos (unidecode)
    tokens = [unidecode(t) for t in tokens]
  elif palavras:
    tokens = []

    for idx, palavra in enumerate(palavras):
      token = gera_tokens(texto=palavra,
                          no_stemming=no_stemming)
      tokens.extend(token)

  return tokens


def get_representacao(arquivo_caminho: str,
                      no_stemming: bool = False) -> Dict[str, Dict[str, int]]:
  """
  Gera a representação do arquivo com a frequência de cada token.

  Exemplo retorno:
  {
    "token": {
      "relevancia": 10,
      "frequencia": 7
    }, ...
  }
  """
  from collections import Counter

  f = open(arquivo_caminho, 'r')
  linhas = f.readlines()
  f.close()

  # remove url das imagens
  linhas.pop(22)  # habilidades
  linhas.pop(18)
  linhas.pop(14)
  linhas.pop(10)
  linhas.pop(6)  # passiva
  linhas.pop(2)  # retrato

  texto = '\n'.join(linhas)

  titulo_tokens = gera_tokens(texto=linhas[0],
                              no_stemming=no_stemming)

  tags_tokens = gera_tokens(texto=linhas[1],
                            no_stemming=no_stemming)

  tokens = gera_tokens(texto=texto,
                       no_stemming=no_stemming)

  frequencias = dict(Counter(tokens))

  for chave in frequencias:
    frequencias[chave] = {
        'relevancia': 1,
        'frequencia': frequencias[chave]
    }
  
  # relevancias para nome, titulo, e tags
  titulo_relevancia = 10
  tags_relevancia = 5

  for token in titulo_tokens:
    if token in frequencias:
      frequencias.get(token)['relevancia'] = titulo_relevancia
  
  for token in tags_tokens:
    if token in frequencias:
      frequencias.get(token)['relevancia'] = tags_relevancia

  return frequencias


def gera_representacoes(de_caminho: str,
                        para_caminho: str) -> None:
  """
  Gera arquivos de representações com as frequências de cada token.
  """
  import os, json, ntpath
  from glob import glob

  de_caminho = os.path.join(de_caminho, '*.txt')
  
  arquivos = glob(de_caminho)

  for arquivo in arquivos:
    arquivo_nome = ntpath.basename(arquivo)
    arquivo_nome = arquivo_nome.rsplit('.', 1)[0] + '.json'
    representacao = get_representacao(arquivo)
    with open(os.path.join(para_caminho, arquivo_nome), "w") as outfile:
      json.dump(representacao, outfile)
    print(f"{arquivo} processado.")

  print('Arquivos de representação gerados.')


## Carrega arquivos

Caso os arquivos tenham sido gerados anteriormente.

In [ ]:
from google.colab import files

# league_of_legends.zip
for f in files.upload():
  print('Arquivo:', f)
  _ext = f.rsplit('.', 1)[1]
  if _ext == 'zip':
    !unzip -q {f}

Saving league_of_legends.zip to league_of_legends.zip
Arquivo: league_of_legends.zip


## Gera representação

In [ ]:
gera_representacoes(BASE_FOLDER, REPRESENTACAO_FOLDER)

league_of_legends/Ekko.txt processado.
league_of_legends/TahmKench.txt processado.
league_of_legends/Lucian.txt processado.
league_of_legends/Fiora.txt processado.
league_of_legends/Kindred.txt processado.
league_of_legends/Pantheon.txt processado.
league_of_legends/Samira.txt processado.
league_of_legends/Jinx.txt processado.
league_of_legends/Jax.txt processado.
league_of_legends/Riven.txt processado.
league_of_legends/Gwen.txt processado.
league_of_legends/MasterYi.txt processado.
league_of_legends/Yorick.txt processado.
league_of_legends/Diana.txt processado.
league_of_legends/Khazix.txt processado.
league_of_legends/Alistar.txt processado.
league_of_legends/Corki.txt processado.
league_of_legends/Aphelios.txt processado.
league_of_legends/Velkoz.txt processado.
league_of_legends/Illaoi.txt processado.
league_of_legends/Malzahar.txt processado.
league_of_legends/Senna.txt processado.
league_of_legends/Volibear.txt processado.
league_of_legends/Fizz.txt processado.
league_of_legends

### Gera zip

In [ ]:
!zip -r {REPRESENTACAO_FOLDER}.zip {REPRESENTACAO_FOLDER}

  adding: representacao/ (stored 0%)
  adding: representacao/Yone.json (deflated 89%)
  adding: representacao/Aphelios.json (deflated 88%)
  adding: representacao/Ivern.json (deflated 89%)
  adding: representacao/TwistedFate.json (deflated 89%)
  adding: representacao/Warwick.json (deflated 89%)
  adding: representacao/Kaisa.json (deflated 89%)
  adding: representacao/Maokai.json (deflated 88%)
  adding: representacao/Xayah.json (deflated 89%)
  adding: representacao/Karthus.json (deflated 88%)
  adding: representacao/Syndra.json (deflated 89%)
  adding: representacao/Bard.json (deflated 89%)
  adding: representacao/Samira.json (deflated 89%)
  adding: representacao/Poppy.json (deflated 88%)
  adding: representacao/Riven.json (deflated 89%)
  adding: representacao/LeeSin.json (deflated 89%)
  adding: representacao/JarvanIV.json (deflated 89%)
  adding: representacao/Jinx.json (deflated 89%)
  adding: representacao/Vayne.json (deflated 89%)
  adding: representacao/Caitlyn.json (deflated

### Carrega arquivos

Caso as representações tenham sido feitas anteriormente.

In [ ]:
from google.colab import files

# representacao.zip
for f in files.upload():
  print('Arquivo:', f)
  _ext = f.rsplit('.', 1)[1]
  if _ext == 'zip':
    !unzip -q {f}

# Etapa 3: Recuperação dos documentos ordenados com consulta do usuário

## Funções

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def calcula_idfs(arquivos_pasta: str,
                 tokens: List[str]
                 ) -> Tuple[Dict[str, float], Dict[str, float]]:
  """
  Calcula idf de cada palavra em uma lista em relação a um diretório de jsons.
  """
  from glob import glob
  import os, json

  docs_caminhos = glob(os.path.join(arquivos_pasta, '*.json'))
  n_docs = len(docs_caminhos)
  n_docs_com_termos = [0 for _ in tokens]
  max_freq = {}
  for caminho in docs_caminhos:
    with open(caminho) as arquivo:
      representacao = json.load(arquivo)
      for idx, token in enumerate(tokens):
        if token in representacao:
          n_docs_com_termos[idx] += 1
          if not max_freq.get(token) or \
              representacao.get(token).get('frequencia') > max_freq.get(token):
            max_freq[token] = representacao.get(token).get('frequencia')
  
  idfs = {}
  for idx, n_docs_com_termo in enumerate(n_docs_com_termos):
    if n_docs_com_termo != 0:
      idfs[tokens[idx]] = float(n_docs / n_docs_com_termo)
    else:
      idfs[tokens[idx]] = 0.0
  return idfs, max_freq


def calcula_pesos(representacao: Dict[str, Dict[str, int]],
                  tokens: List[str],
                  idfs: Dict[str, float],
                  max_freq: Dict[str, float]) -> Dict[str, float]:
  """
  Calcula pesos de cada token em um documento. w = freq * idf
  """
  import os, json

  pesos = {}

  for token in tokens:
    if token in representacao:
      frequencia = representacao.get(token).get('frequencia')
      pesos[token] = frequencia * idfs[token]
      # normalizando
      pesos[token] /= max_freq[token] * idfs[token]
    else:
      pesos[token] = 0.0

  return pesos


def calcula_similaridade(representacao: Dict[str, Dict[str, int]],
                         pesos: Dict[str, float],
                         p: int = 3) -> float:
  """
  Calcula similaridade entre doc e busca de acordo com os pesos.
  Fórmula: 1-( (wl1^p(1-w1)^p+wl2^p(1-w2)^p+...) / (wl1^p+wl2^p+...) )^(1/p)
  """
  import math

  somatorio_pesos = 0
  for token in pesos:
    somatorio_pesos += pesos.get(token)
  if somatorio_pesos == 0:
    return float('-inf')
  
  somatorio_numerador = 0.0
  for token in pesos:
    peso_relevancia = 1
    if token in representacao:
      peso_relevancia = representacao.get(token).get('relevancia')
    somatorio_numerador += \
      math.pow(peso_relevancia, p) * math.pow(1.0 - pesos.get(token), p)
  
  somatorio_denominador = 1.0
  for token in pesos:
    peso_relevancia = 1
    if token in representacao:
      peso_relevancia = representacao.get(token).get('relevancia')
    somatorio_denominador += math.pow(peso_relevancia, p)

  return 1 - math.pow(somatorio_numerador / somatorio_denominador, 1 / p)


def calcula_similaridades(pesquisa: str,
                          arquivos_pasta: str,
                          ) -> List[Dict[str,
                                         Union[str, float, Dict[str, float]]]]:
    """
    Calcula similaridade de todos os arquivos com a pesquisa.
    Exemplo retorno:
    [{
        "id": "id",
        "pesos": {
          "termo1": 0.5,
          "termo2": 0.7, ...
        },
        "similaridade": 0.6
    }, ...]
    """
    from glob import glob
    import os, re, json

    # separa a pesquisa por espaços, vírgulas, ou pontos
    # levando em consideração possíveis números reais
    termos_pesquisa = re.split("\s|(?<!\d)[,.](?!\d)", pesquisa)

    tokens = gera_tokens(palavras=termos_pesquisa)

    idfs, max_freq = calcula_idfs(arquivos_pasta=arquivos_pasta,
                                  tokens=tokens)

    docs_caminhos = glob(os.path.join(arquivos_pasta, '*.json'))

    similaridades = []
    for caminho in docs_caminhos:
      with open(caminho) as arquivo:
        similaridade = {}
        representacao = json.load(arquivo)
        similaridade['id'] = os.path.basename(caminho).rsplit('.', 1)[0]
        pesos = calcula_pesos(representacao=representacao,
                               tokens=tokens,
                               idfs=idfs,
                               max_freq=max_freq)
        similaridade['pesos'] = pesos
        similaridade['similaridade'] =\
          calcula_similaridade(representacao=representacao,
                               pesos=pesos)
        similaridades.append(similaridade)

    return similaridades


def resultado(busca: str,
              representacoes_caminho: str
              ) -> List[Dict[str, Any]]:
  """
  Retorna os resultados da busca.
  """
  similaridades = calcula_similaridades(busca, representacoes_caminho)

  return sorted(similaridades,
                key=lambda key: key.get('similaridade'),
                reverse=True)


def mostra_titulo(titulo: Union[str, bool] = False) -> None:
  from IPython.display import Image, display, Markdown

  if not bool(titulo):
    display(Markdown(f'A busca precisa ter 3 ou mais caracteres.'))
  else:
    display(Markdown(f'# Pesquisa: "{titulo}"'))


def mostra_resultados(resultados: List[Dict[str, Any]],
                      arquivos_caminho: str,
                      arquivos_url: str) -> None:
  """
  Mostra os resultados na célula.
  """
  from IPython.display import Image, display, Markdown
  import os

  contador = 0
  for resultado in resultados:
    if resultado.get('similaridade') != float('-inf'):
      contador += 1
      id = resultado.get('id')

      f = open(os.path.join(arquivos_caminho, id + '.txt'), 'r')
      linhas = f.readlines()
      f.close()

      retrato = linhas[2]

      display(Markdown(f'##\# {contador} - {linhas[0]}'))
      display(Image(url=retrato))

      url = arquivos_url + '/' + id + '.txt'
      
      print('Similaridade:', resultado.get('similaridade'))
      print(url)
      print('Resumo:', linhas[24])


def pesquisa(busca: str,
             representacao_caminho: str,
             arquivos_caminho: str,
             arquivos_url: str) -> None:
  """
  Faz a pesquisa e mostra os resultados.
  """
  if not bool(busca) or len(busca) < 3:
    mostra_titulo(False)
  else:
    resultado_dict = resultado(busca, representacao_caminho)
    mostra_titulo(busca)
    print()
    mostra_resultados(resultado_dict, arquivos_caminho, arquivos_url)



## Variáveis

In [ ]:
arquivos_url = 'https://raw.githubusercontent.com/pvbernhard/recuperacao_informacao/master/league_of_legends'

## Input do usuário

In [ ]:
#@title Pesquisa

texto_busca = 'teste'  #@param {type: "string"}

## Pesquisa

In [ ]:
pesquisa(texto_busca, REPRESENTACAO_FOLDER, BASE_FOLDER, arquivos_url)